In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.1
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [3]:
def synthetic_data(true_w, true_b, num_input):
    X = torch.normal(0, 1, (num_input, true_w))
    y = torch.matmul(X, true_w) + true_b
    y += torch.normal(0, 1, y.shape)
    return X, y.reshape((-1, 1))

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [5]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [6]:
next(iter(data_iter))

[tensor([[-1.3646, -0.1049],
         [-1.3804, -0.4122],
         [-0.4673,  0.4943],
         [ 0.2568,  0.3838],
         [ 0.0430,  0.4445],
         [-1.4586, -1.1004],
         [-0.4159, -1.1721],
         [ 0.4057,  1.2182],
         [ 1.6049, -0.7431],
         [ 1.9287,  0.2537]]),
 tensor([[1.7106],
         [2.7437],
         [1.4823],
         [3.3156],
         [2.6661],
         [4.9270],
         [7.2531],
         [0.7726],
         [9.8420],
         [7.0981]])]

In [7]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [8]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [9]:
loss = nn.MSELoss()

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [11]:
num_epochs = 3
for epoch in  range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000237
epoch 2, loss 0.000100
epoch 3, loss 0.000100


In [13]:
w = net[0].weight.data
print(w.shape, true_w.shape)
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

torch.Size([1, 2]) torch.Size([2])
w的估计误差： tensor([3.6240e-05, 6.8259e-04])
b的估计误差： tensor([-0.0003])
